In [12]:
import cv2
import numpy as np
from pathlib import Path
from scipy import stats
import pandas as pd
from webcolors import rgb_to_name, hex_to_rgb

def read_image(input_path: Path) -> np.ndarray:
    """
    Read the image from the given path using OpenCV.
    """
    img = cv2.imread(str(input_path), cv2.IMREAD_UNCHANGED)
    return img

def save_image(output_path: Path, img: np.ndarray) -> None:
    """
    Save the processed image to the given path using OpenCV.
    """
    cv2.imwrite(str(output_path), img)

def calculate_most_common_color(square: np.ndarray) -> np.ndarray:
    """
    Calculate the most common color of the given square.
    """
    reshaped_square = square.reshape(-1, square.shape[-1])
    mode_color = stats.mode(reshaped_square, axis=0)[0]
    return mode_color

def draw_square(image: np.ndarray, 
                x: int, 
                y: int, 
                square_size: int, 
                border_color: tuple, 
                fill_color: tuple) -> None:
    """
    Draw a square with a specified border and 
    fill color on the image at the specified location.
    """
    image[y:y+square_size, x:x+square_size] = fill_color
    image[y:y+1, x:x+square_size] = border_color
    image[y+square_size-1:y+square_size, x:x+square_size] = border_color
    image[y:y+square_size, x:x+1] = border_color
    image[y:y+square_size, x+square_size-1:x+square_size] = border_color

def get_html_color_name(color: tuple) -> str:
    """
    Get the HTML color name for a given RGB color.
    """
    hex_color = '#{:02x}{:02x}{:02x}'.format(color[0], color[1], color[2])
    try:
        color_name = rgb_to_name(hex_to_rgb(hex_color))
    except ValueError:
        color_name = hex_color
    return color_name

def process_image(image: np.ndarray, square_size: int) -> np.ndarray:
    """
    Process the image by replacing each square with its most common color 
    and creating a new image with white or light gray squares.
    """
    output_image = np.zeros_like(image)
    height, width, _ = image.shape
    color_mapping = []

    for i in range(0, height, square_size):
        for j in range(0, width, square_size):
            square = image[i:i+square_size, j:j+square_size]
            most_common_color = calculate_most_common_color(square)
            html_color_name = get_html_color_name(tuple(most_common_color[0][:3]))
            color_mapping.append({"coordinates": (j, i), 
                                  "color": most_common_color.tolist(), 
                                  "color_name": html_color_name})

            if html_color_name == "#f7f7f7":
                fill_color = (192, 192, 192, 255)
            else:
                fill_color = (255, 255, 255, 255)

            draw_square(output_image, 
                        j, 
                        i, 
                        square_size, 
                        (192, 192, 192, 255), 
                        fill_color)

    return output_image, color_mapping

def write_csv(output_path: Path, color_mapping: list) -> None:
    """
    Write the color mapping to a CSV file.
    """
    df = pd.DataFrame(color_mapping)
    df.to_csv(output_path, index=False)



In [16]:
def main():
    base_folder = Path("/Users/senthilgandhi/Dropbox/parent/workspace/pixel_puzzle_math")
    input_path = base_folder / "input_image.png"
    output_path = base_folder / "output_image.png"
    csv_path = base_folder / "color_mapping.csv"
    square_size = 16

    input_image = read_image(input_path)
    processed_image, color_mapping = process_image(input_image, square_size)
    save_image(output_path, processed_image)
    write_csv(csv_path, color_mapping)

    df = pd.DataFrame(color_mapping)
    print(df.head())
    unique_colors = df['color_name'].nunique()
    non_white_squares = df[df['color_name'] != '#f7f7f7'].shape[0]

    print(f"Unique colors encountered: {unique_colors}")
    print(f"Total number of non-white squares: {non_white_squares}")

if __name__ == "__main__":
    main()


  coordinates                   color color_name
0      (0, 0)  [[247, 247, 247, 255]]    #f7f7f7
1     (16, 0)  [[247, 247, 247, 255]]    #f7f7f7
2     (32, 0)  [[247, 247, 247, 255]]    #f7f7f7
3     (48, 0)  [[247, 247, 247, 255]]    #f7f7f7
4     (64, 0)  [[247, 247, 247, 255]]    #f7f7f7
Unique colors encountered: 711
Total number of non-white squares: 2474
